In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sqlalchemy import create_engine
username = "multi01"
password = "1111"
host = "ec2-15-168-111-154.ap-northeast-3.compute.amazonaws.com"
database_name = "recipe_db"

db_connection_str = f'mysql+pymysql://{username}:{password}@{host}/{database_name}'
db_connection = create_engine(db_connection_str)

In [8]:
keyword = "감바스" # from cnn result

## 재료와 카테고리 가져오기

In [6]:
def extract_posts(keyword):
    # 각 게시물 별 평점의 평균값 구하기
    query = '''
    SELECT *
    FROM recipe_db.posts_rating;
    '''
    r_df = pd.read_sql(query, con=db_connection)
    # 'id'와 'user_id' 열 제거
    r_df.drop(['id', 'user_id'], axis=1, inplace=True)
    
    # 'post_id' 열을 기준으로 데이터프레임 정렬
    r_df.sort_values(by='post_id', inplace=True)
    # 'post_id'별 'score'의 합과 개수 계산
    post_id_summary = r_df.groupby('post_id')['score'].agg(['sum', 'count']).reset_index()
    
    # 'score'의 합을 'count'로 나누어 평균 계산
    post_id_summary['mean'] = post_id_summary['sum'] / post_id_summary['count']
    
    # 새로운 데이터프레임 생성
    mean_df = post_id_summary[['post_id', 'mean']]
    query2 = f'''
    SELECT id
    FROM recipe_db.posts_post
    WHERE INSTR(title, '{keyword}') > 0;
    '''
    e_df = pd.read_sql(query2, con=db_connection)
    id_list = e_df['id'].tolist()
    # id_list에 있는 각 post_id에 해당하는 평균값 추출
    id_means = mean_df[mean_df['post_id'].isin(id_list)]

    # id_means가 비어 있는지 확인하고 예외처리 추가
    if not id_means.empty:
        max_mean_post_id = id_means.loc[id_means['mean'].idxmax()]['post_id']

        query3 = '''
        SELECT C.ingred_name, A.post_id
        FROM recipe_db.posts_recipeingred A 
        LEFT JOIN recipe_db.posts_post B ON A.post_id = B.id
        LEFT JOIN recipe_db.posts_ingred C ON A.ingred_id = C.id;
        '''
        i_df = pd.read_sql(query3, con=db_connection)
        # post_id별 ingred_name 리스트로 모으기
        grouped_ingreds = i_df.groupby('post_id')['ingred_name'].apply(list).reset_index()
        result = grouped_ingreds[grouped_ingreds['post_id'] == max_mean_post_id]['ingred_name'].values[0]
    
        query4 = '''
        SELECT id, cate1, cate2, cate3
        FROM recipe_db.posts_post;
        '''
        cate_df = pd.read_sql(query4, con=db_connection)
        result2 = cate_df[cate_df['id'] == max_mean_post_id][['cate1', 'cate2', 'cate3']]
        result_list = result2.values.flatten().tolist()
        return result, result_list
    else:
    # id_means가 비어 있을 때 처리할 내용 추가
        return None, None  # 또는 다른 처리 방법을 선택하여 반환

In [9]:
extract_posts(keyword)

(['올리브오일', '마늘', '후추', '파슬리', '칵테일새우', '페퍼론치노', '소금', '바게트'],
 ['양식', '술안주', '해물'])